## Team 10 program Updated v1
### Anthony Ung

#### Modifications Made By AU Before Profiling
- Added the DB API from AU's program
- Changed file path to point inside the Github Repo

- Fixed bug where Team 10 did not account for 2024 being a Leap Year
- Fixed bug where Team 10 restocked all products every day

- Team 10 did not do anything at all with the special product categories or the association rules.
  - AU left this as is and will fix in the v2 Program
- Team 10 limited customers to selecting 5 products.
  - AU left this as is and will fix in the v2 Program

In [1]:
import sqlite3 as lite

'''
    This class provides one common point of interaction with my team's database.
    Everything that writes to the database uses this API.
'''
class db:
    
    def __init__(self, name):
        self.name = rf"{name}"

    def connect(self):
        self.con = lite.connect(self.name)
        self.cur = self.con.cursor()

    def build_table(self, name):      
        self.execute_sql(f'DROP TABLE IF EXISTS {name}')
        self.execute_sql(TABLE_DEFINITIONS[name])
    
    def execute_sql(self, sql, print_results=False):
        if print_results == True:
            results = self.cur.execute(sql).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql)

    def execute_sql_values(self, sql, values):
        self.cur.execute(sql, values)

    def commit(self):
        self.con.commit()

    def close(self):
        self.con.commit()
        self.con.close()

In [7]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
import datetime

# Constants
DAILY_CUSTOMER_MIN, DAILY_CUSTOMER_MAX = 1070, 1100
WEEKEND_EXTRA_CUSTOMERS = 50
PRICE_MULTIPLIER = 1.15
CASE_SIZE = 12
INITIAL_DAYS_SUPPLY = 3
INITIAL_MILK_SUPPLY = 1.5

# Load product data
product_file = r"Products1.txt"
PRODUCTS = pd.read_csv(product_file, delimiter="|", encoding="ISO-8859-1")
PRODUCTS["BasePrice"] = PRODUCTS["BasePrice"].str.replace("$", "", regex=False).astype(float)
PRODUCT_LOOKUP = PRODUCTS.set_index("SKU")["BasePrice"].to_dict()

for sku in PRODUCT_LOOKUP:
    PRODUCT_LOOKUP[sku] = round(PRODUCT_LOOKUP[sku] * PRICE_MULTIPLIER, 2)

SKUs = PRODUCTS["SKU"].values

# Initialize inventory
INVENTORY = {sku: max(INITIAL_DAYS_SUPPLY * 40, CASE_SIZE) for sku in SKUs}
INVENTORY["Milk"] = max(INITIAL_MILK_SUPPLY * 40, CASE_SIZE)
total_cases_ordered = defaultdict(int)

# Generate sales data
sales_data = []
start_date = pd.Timestamp("2024-01-01")

start_dt = datetime.datetime(year=2024, month=1, day=1)
end_dt = datetime.datetime(year=2024, month=12, day=31)

current_dt = start_dt

TABLE_DEFINITIONS = {
    'sales_transactions': \
            'CREATE TABLE sales_transactions(' \
                    'date TEXT, ' \
                    'customer_number INT, ' \
                    'sku INT, ' \
                    'salesPrice REAL, ' \
                    'items_left INT, ' \
                    'cases_ordered INT)'
}

db_10 = db('grocery_team_10.db')
db_10.connect()
db_10.build_table('sales_transactions')

while(current_dt <= end_dt):
    daily_customers = random.randint(DAILY_CUSTOMER_MIN, DAILY_CUSTOMER_MAX)
    if current_dt.weekday() >= 5:
       daily_customers += WEEKEND_EXTRA_CUSTOMERS

    if current_dt.weekday() not in (1, 3, 5):
        for sku in INVENTORY:            
            if sku == "Milk":
                threshold = INITIAL_MILK_SUPPLY * 40  # 1.5 days of supply for Milk
            else:
                threshold = 99999999 # Should never happen in practice
                
            if INVENTORY[sku] < threshold:
                # Calculate how many cases to order
                cases_to_order = ((threshold - INVENTORY[sku]) // CASE_SIZE) + 1
                INVENTORY[sku] += cases_to_order * CASE_SIZE
                total_cases_ordered[sku] += cases_to_order
            
    else:
        for sku in INVENTORY:
            if sku == "Milk":
                threshold = INITIAL_MILK_SUPPLY * 40  # 1.5 days of supply for Milk
            else:
                threshold = INITIAL_DAYS_SUPPLY * 40  # 3 days of supply for other products

            if INVENTORY[sku] < threshold:
                # Calculate how many cases to order
                cases_to_order = ((threshold - INVENTORY[sku]) // CASE_SIZE) + 1
                INVENTORY[sku] += cases_to_order * CASE_SIZE
                total_cases_ordered[sku] += cases_to_order

    date_str = current_dt.strftime("%Y%m%d")
    
    for customer_id in range(daily_customers):
        purchased_items = np.random.choice(SKUs, size=random.randint(1, 5), replace=True)

        for sku in purchased_items:
            if INVENTORY[sku] > 0:
            # Item is available, so reduce the inventory by 1
                INVENTORY[sku] -= 1
    
                try:
                    db_10.execute_sql_values('insert into sales_transactions values (?, ?, ?, ?, ?, ?)',
                                            (date_str,customer_id,sku,PRODUCT_LOOKUP[sku], INVENTORY[sku], total_cases_ordered[sku]))
                except Exception as err:
                    print("Error writing to sales_transactions database table", err)
            
            else:
                # If item is out of stock, substitute with another available item
                substitutes = [s for s in skus if inventory[s] > 0]
                if substitutes:
                    sku = random.choice(substitutes)
                    INVENTORY[sku] -= 1

                    try:
                        db_10.execute_sql_values('insert into sales_transactions values (?, ?, ?, ?, ?, ?)',
                                                (date_str,customer_id,sku,PRODUCT_LOOKUP[sku], INVENTORY[sku], total_cases_ordered[sku]))
                    except Exception as err:
                        print("Error writing to sales_transactions database table", err)

    db_10.commit()
    current_dt += datetime.timedelta(days=1)
        
db_10.close()
print('Finished')


Finished
